In [1]:
import logging
import math
import numpy as np
import os
import sys
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional

import datasets
from datasets import load_dataset, load_metric

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_MASKED_LM_MAPPING,
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

import gpn.mlm
import transformers
from transformers import AutoModelForMaskedLM
import os
from torchinfo import summary
import torch
import datasets
from datasets import load_dataset, load_metric
os.environ['CUDA_VISIBLE_DEVICES']='1'
model = AutoModelForMaskedLM.from_pretrained('./GPN_finetune/checkpoint-2000000/')

In [2]:
raw_datasets = load_dataset('parquet', 
                            data_files='/home/amber/multitask_RNA/data/gpn/test/hg38.test.512.256.parquet', 
                            cache_dir= None)

Using custom data configuration default-b4c1d8f48d8bcb2e
Found cached dataset parquet (/home/amber/.cache/huggingface/datasets/parquet/default-b4c1d8f48d8bcb2e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained('gonzalobenegas/gpn-arabidopsis')
def tokenize_function(examples):
            # Remove empty lines
            examples[text_column_name] = [
                line for line in examples[text_column_name] if len(line) > 0 and not line.isspace()
            ]
            res = tokenizer(
                examples[text_column_name],
                #padding=padding,
                #truncation=True,
                #max_length=max_seq_length,
                # We use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
                # receives the `special_tokens_mask`.
                #return_special_tokens_mask=True,
                return_attention_mask=False,
                return_token_type_ids=False,
            )
            #res["special_tokens_mask"] = np.char.islower(np.vstack([np.array(list(seq)) for seq in examples[text_column_name]]))
            #print(examples[text_column_name], res["special_tokens_mask"])
            #raise Exception("debug eval")
            return res

In [4]:
column_names = raw_datasets["train"].column_names
text_column_name = "seq"
tokenized_datasets = raw_datasets.map(
                tokenize_function,
                batched=True,
                num_proc=1,
                remove_columns=column_names, #[text_column_name],
                load_from_cache_file=True,
                desc="Running tokenizer on dataset line_by_line",
            )

Loading cached processed dataset at /home/amber/.cache/huggingface/datasets/parquet/default-b4c1d8f48d8bcb2e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-cb6b17a86ff69d0c.arrow


In [11]:
len(tokenized_datasets['train'][0]['input_ids'])

512

In [12]:
test_input = tokenized_datasets['train'][0]['input_ids']

In [15]:
import torch.nn.functional as F
test_embed = F.one_hot(torch.tensor(test_input), num_classes=512).float()

In [ ]:
test_input[0]

In [17]:
test_embed[0]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [24]:
raw_datasets['train'][0]

{'start': 87732014,
 'end': 87732526,
 'chrom': 'chr8',
 'strand': '-',
 'seq': 'ATAATTTTTCATGTTCTAATCTCATTTCCAGCACAATCTCATTTTTATGACGCAAATTGTAAGTCTTGATAGAAACGTGCAATTTGTAGCTGTCTGTTTTATTTTTGATCAGATTTTCATCTTGACCTATATCTCAGTGAAATGTTATCTTCTATTGTAGCAAGTTGTTTATTTGCTATACTTCTAAGTGGCTGGAGAGGCCCTAGGATAAATTATGTGGTAAATTATTGGCCTTCAGTATCAAACATGtctttcagtaagcatttattaagtatttctatgcttcaggcaatatggctggcacttatactcaagcagcttacaggttacaggaagaaatatgtgactagattattatgataacataataagcactactctagaagtatggacaacatcctataaatagcaagagctcctaactgagagtgtgtgttagagaaggtttcctggCTGTTTTAAGATAAGACCTAAAGTTTtcttacttatttttatacctacatcgagacctt',
 '__index_level_0__': 906959}

In [12]:
summary(model,input_size=(2,512),dtypes= [torch.long])

Layer (type:depth-idx)                             Output Shape              Param #
ConvNetForMaskedLM                                 [2, 512, 6]               --
├─ConvNetModel: 1-1                                [2, 512, 512]             --
│    └─OneHotEmbedding: 2-1                        [2, 512, 512]             --
│    └─Sequential: 2-2                             [2, 512, 512]             --
│    │    └─ConvLayer: 3-1                         [2, 512, 512]             2,624,512
│    │    └─ConvLayer: 3-2                         [2, 512, 512]             2,624,512
│    │    └─ConvLayer: 3-3                         [2, 512, 512]             2,624,512
│    │    └─ConvLayer: 3-4                         [2, 512, 512]             2,624,512
│    │    └─ConvLayer: 3-5                         [2, 512, 512]             2,624,512
│    │    └─ConvLayer: 3-6                         [2, 512, 512]             2,624,512
│    │    └─ConvLayer: 3-7                         [2, 512, 512]         

In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained('gonzalobenegas/gpn-arabidopsis')